In [28]:
pip install tabulate

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from tabulate import tabulate


# URL de la página web que deseamos hacer scraping
url = 'https://elcomercio.pe/archivo/todas/2023-10-02/'

# Realizamos una solicitud GET a la página web
response = requests.get(url)

# Verificamos si la solicitud fue exitosa (código de respuesta 200)
if response.status_code == 200:
    # Parseamos el contenido HTML de la página web
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontramos todos los elementos con la etiqueta 'span' que contienen la fecha y hora
    Fecha_Hora = soup.find_all('p', class_="story-item__date font-thin ml-5 text-xs text-gray-300 md:mt-5 md:ml-0")

    # Encontramos todos los elementos con la etiqueta 'h2' que contienen los títulos de las noticias
    titulos = soup.find_all('h2')

    # Encontramos todos los elementos con la etiqueta 'div' que contienen el autor
    Author = soup.find_all('div', class_="hidden")

    # Crear DataFrames para cada categoría
    df_fecha_hora = pd.DataFrame({'Fecha_Hora': [fechaH.text.strip() for fechaH in Fecha_Hora]})
    df_titulos = pd.DataFrame({'Títulos': [titulo.text.strip() for titulo in titulos]})
    df_author = pd.DataFrame({'Autor': [Authors.text.strip() for Authors in Author]})

    # Combina los DataFrames
    df = pd.concat([df_fecha_hora, df_titulos, df_author], axis=1)

    # Agregar una columna de numeración a los registros
    df.insert(0, 'Número', range(1, len(df) + 1))

    # Imprimir el DataFrame de forma ordenada en una tabla con bordes y numeración
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False, numalign='left'))

    # Crear un nuevo archivo Excel
    nombre_archivo = 'web_scraping_elcomercio_2023-10-02.xlsx'
    wb = Workbook()
    ws = wb.active
    ws.title = 'Resultados'

    # Agregar los datos del DataFrame al archivo Excel
    for row in dataframe_to_rows(df, index=False, header=True):
        ws.append(row)

    # Guardar el archivo Excel
    wb.save(nombre_archivo)

    print(f'Se han guardado los resultados en "{nombre_archivo}" con formato de tabla.')
else:
    print('Error al obtener la página:', response.status_code)
